In [2]:
# torch, autograd and backpropagation
# https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/


import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

L = 10 - d

print("The grad fn for a is", a.grad_fn)
print("The grad fn for d is", d.grad_fn)

The grad fn for a is None
The grad fn for d is <AddBackward0 object at 0x7f0b2d7068e0>


In [3]:
b.is_leaf

False

In [4]:
w1.is_leaf

True

In [5]:
b.grad_fn

In [6]:
import torch 
a = torch.ones(5)
a.requires_grad = True

b = 2*a

b.retain_grad()   # Since b is non-leaf and it's grad will be destroyed otherwise.

c = b.mean()

c.backward()

print(a.grad, b.grad)

tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [7]:
# Redo the experiment but with a hook that multiplies b's grad by 2. 
a = torch.ones(5)

a.requires_grad = True

b = 2*a

b.retain_grad()

b.register_hook(lambda x: print('Hello Im a tensor backward hook reporting on x:', x))  

b.mean().backward() 


print(a.grad, b.grad)

Hello Im a tensor backward hook reporting on x: tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])
tensor([0.4000, 0.4000, 0.4000, 0.4000, 0.4000]) tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])


In [8]:
# Playing with hooks (https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/)

import torch 
import torch.nn as nn

class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
   
  
  def forward(self, x):
    x = self.relu(self.conv(x))
    x.register_hook(lambda grad : torch.clamp(grad, min = 0))     #No gradient shall be backpropagated 
                                                                  #conv outside less than 0
      
    # print whether there is any negative grad
    x.register_hook(lambda grad: print("Gradients less than zero:", bool((grad < 0).any())))  
    return self.fc1(self.flatten(x))
  

net = myNet()

for name, param in net.named_parameters():
  # if the param is from a linear and is a bias
  if "fc" in name and "bias" in name:
    param.register_hook(lambda grad: torch.zeros(grad.shape))


out = net(torch.randn(1,3,8,8)) 

(1 - out).mean().backward()

print("The biases are", net.fc1.bias.grad)     #bias grads are zero

Gradients less than zero: False
The biases are tensor([0., 0., 0., 0., 0.])


In [9]:
# The Forward Hook for Visualising Activations 

visualisation = {}

inp = torch.randn(1,3,8,8)

def hook_fn(m, i, o):
  visualisation[m] = o 
  
net = myNet()

for name, layer in net._modules.items():
  print('net._modules.items().__class__.__name__:', net._modules.items())
  layer.register_forward_hook(hook_fn)
  
out = net(inp) 

net._modules.items().__class__.__name__: odict_items([('conv', Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))), ('relu', ReLU()), ('fc1', Linear(in_features=160, out_features=5, bias=True))])
net._modules.items().__class__.__name__: odict_items([('conv', Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))), ('relu', ReLU()), ('fc1', Linear(in_features=160, out_features=5, bias=True))])
net._modules.items().__class__.__name__: odict_items([('conv', Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))), ('relu', ReLU()), ('fc1', Linear(in_features=160, out_features=5, bias=True))])


In [10]:
inp = torch.randn(1,3,8,8)

In [11]:
type(inp.size()[2])

int

In [12]:
w=torch.randn(2,2)

In [13]:
w

tensor([[ 0.3264, -0.1605],
        [ 0.6966,  0.4734]])

In [14]:
te=torch.randn(2,2)

In [15]:
x = (torch.rand(2, 2) * 10).int()

In [16]:
x

tensor([[3, 6],
        [7, 1]], dtype=torch.int32)

In [17]:
te = (torch.rand(2, 2) * 10).int()

In [18]:
te

tensor([[9, 5],
        [0, 4]], dtype=torch.int32)

In [19]:
te * x

tensor([[27, 30],
        [ 0,  4]], dtype=torch.int32)

In [20]:
(1 -te)

tensor([[-8, -4],
        [ 1, -3]], dtype=torch.int32)

In [21]:
(1-te)*x

tensor([[-24, -24],
        [  7,  -3]], dtype=torch.int32)

In [ ]:
nn = {
    '0-0-0-0': [1,2,3,4,5,6,7,8,9,0],
    '0-1-0-1': [0,.9789,8,7,6,5,4,3,2,1]
}

import pandas as pd

df = pd.DataFrame(nn)

df.head()

df.to_csv("bblob.csv", sep=',', index=False)

In [3]:
mylist = [1,2,5,6,7,8,9]

next((i+1 for i, x in enumerate(mylist) if x >= 18), len(mylist))

7

In [13]:

import pandas as pd
import os

import pathlib


info = {
            'target_epoch_accuracy': [100],
            'epoch': [300], 
            'epoch_loss': [2.3478567],
            'epoch_accuracy': [80],
        }
print(type(info))

filename= 'XORNET-ADAM-0.03-blablab.csv'

file = pathlib.Path(filename)
if file.exists ():

    df = pd.read_csv(filename)
    df2 = pd.DataFrame(info)
    result = pd.concat([df, df2])
    result.to_csv(filename, sep=',', index=None)
else:

    df = pd.DataFrame(info)
    df.to_csv(filename, sep=',', index=None)


<class 'dict'>
File exists


In [2]:
[i for i in range(1, 3)]

[1, 2]

In [3]:
num_layers=3
for source_layer in range(0, num_layers-1):
    for target_layer in range(source_layer+1, num_layers):
        print(f'Source layer {source_layer}, target layer {target_layer}')

Source layer 0, target layer 1
Source layer 0, target layer 2
Source layer 1, target layer 2
